### Find Original Distance (GAS-PIPELINES)

In [25]:
import pandas as pd
import numpy as np
import math
import json
import glob


import rasterio
import shapefile
from IPython.display import display

In [26]:
def deg2rad(deg):
    return (deg * (math.pi/180))

In [27]:
def get_distance(p, q):
    lat1, lon1 = p
    lat2, lon2 = q
    #const [lat1, lon1] = pair1, [lat2, lon2] = pair2;
    #const R = 6371; // Radius of the earth in km.

    R = 6371

    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) * math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a));

    return R * c

In [28]:
def find_distance(file_name):
    f = open(file_name)
    data = json.load(f)
    points = []
    # print(data)
    for i in range(len(data['features'])):
        if data['features'][i]['geometry']['type'] == 'Point':
            points.append(data['features'][i]['geometry']['coordinates'])
    
    print(len(points))
    distance = 0
    for i in range(1, len(points)):
        p = points[i - 1]
        q = points[i]

        #distance += math.sqrt(math.pow((p[0] - q[0]), 2) + math.pow((p[1] - q[1]), 2))
        distance += get_distance(p, q)
    return distance

In [29]:
path = 'raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

BoundingBox(left=600000.0, bottom=3090240.0, right=809760.0, top=3300000.0)
600000.0 3090240.0 809760.0 3300000.0


In [30]:
# reading LULC data
area_cover = 'mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

mask_data = area
print(mask_data.shape)
print(np.unique(mask_data, return_counts = True))

mask_data = mask_data.T

(20976, 20976)
(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int16), array([       14,   4922728,   5469572,  24929240,  47784093, 123283586,
       214883874,  18719469]))


In [31]:
index = 0
files = glob.glob(f'OUTPUTS/main_gas.geojson')
# print(files)
for f in files:
    dist = find_distance(f)
    name = files[index].split('.')[0].title()
    print(f'Distance = {dist} Kms, Pipeline = {name}')
    index = index + 1

38
Distance = 206.22126728079346 Kms, Pipeline = Outputs/Main_Gas


In [32]:
MAIN_KILO = dist
print(MAIN_KILO)

road_distance = dist

206.22126728079346


In [38]:
files = glob.glob('OUTPUTS/main_gas_proj.geojson')
print(files)

current_path_list = []
for file_name in files:

    f = open(file_name)
    data = json.load(f)
    # print(data)
    points = []
    for i in range(len(data['features'])):
        if data['features'][i]['geometry']['type'] == 'Point':
            points.append(data['features'][i]['geometry']['coordinates'])
    
    
    
    for i in range(1, len(points)):
        p = points[i - 1]

        current_path_list.append(p)

['OUTPUTS/main_gas_proj.geojson']


In [39]:
pixels_dict = {}
for i in range(len(current_path_list)):

    ext_i, ext_j = current_path_list[i]
    # print(ext_i, ext_j)

    pi, pj = (ext_i - left) / 10, ((ext_j - top) / 10) * -1
    pixels_dict[ext_i, ext_j] = [pi, pj]

In [40]:
classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}

check_pixels = {
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0,
}


total_pxs = len(current_path_list)
for i in range(len(current_path_list)):
    ext_i, ext_j = current_path_list[i]
    pi, pj = pixels_dict[ext_i, ext_j]

    pi, pj = int(pi), int(pj)
    cs = mask_data[pi][pj]

    check_pixels[cs] += 1
    check_pixels[8] += 1


df_dict_pixels = {}
for key, value in check_pixels.items():
        df_dict_pixels[classes[key]] = f'{value} pxs'

for key, value in check_pixels.items():
    check_pixels[key] = np.round(((value / total_pxs) * 100), 2)

df_dict_perc = {}
for key, value in check_pixels.items():
    df_dict_perc[classes[key]] = f'{check_pixels[key]}%'

df_dict_dist = {}
for key, value in check_pixels.items():
    df_dict_dist[classes[key]] = f'{(np.round(check_pixels[key] * road_distance) / 100)} Kms'


list_of_dicts = [df_dict_pixels, df_dict_perc, df_dict_dist,]
route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])
display(route_stats)


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,0 pxs,0 pxs,2 pxs,4 pxs,11 pxs,17 pxs,3 pxs,37 pxs
ROAD PERCENTAGE,0.0%,0.0%,5.41%,10.81%,29.73%,45.95%,8.11%,100.0%
ROAD KILOMETERS,0.0 Kms,0.0 Kms,11.16 Kms,22.29 Kms,61.31 Kms,94.76 Kms,16.72 Kms,206.22 Kms


In [65]:
# path = 'raster.tif' 

# data = rasterio.open(path)
# print(data.bounds)

# extent = data.bounds

# left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
# print(left, bottom, right, top)

BoundingBox(left=300000.0, bottom=1890240.0, right=409800.0, top=2100000.0)
300000.0 1890240.0 409800.0 2100000.0


#### Route-Optimization Analysis

In [47]:
namming_dict = {
    1: 'A* Chebyshev distance',
    2: 'A* Diagonal distance',
    3: 'A* Euclidean distance',
    4: 'A* Manhattan distance',
    5: 'A* Proposed',
    6: 'Dijkstra\'s algorithm',
}

paths_dict = {
    1: 'paths/a-star-chesb',
    2: 'paths/a-star-diag',
    3: 'paths/a-star-euc',
    4: 'paths/a-star-man',
    5: 'paths/a-star-new',
    6: 'paths/dijk-route',
}

shape_dict = {
    1: 'routes-shape/a-star-chesb',
    2: 'routes-shape/a-star-diag',
    3: 'routes-shape/a-star-euc',
    4: 'routes-shape/a-star-man',
    5: 'routes-shape/a-star-new',
    6: 'routes-shape/dijk-route',
}

In [48]:
kilometers = []

In [50]:
for p in range(1, 7):
    path = paths_dict[p]
    path_list = []
    with open(f'OUTPUTS/{path}/path.txt', 'r') as f:
        for point in f:
            path_list.append(point)
    
    # print(len(path_list))
    path_list.reverse()

    ordinates_dict = {}


    for i in range(len(path_list)):
        point = path_list[i].replace('\n', '').split(' ')
        pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

        ext_i, ext_j = left + (pi * 10), top - (pj * 10)

        # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
        ordinates_dict[pi, pj] = [ext_i, ext_j]

    
    ordinates_dict
    ordinates_list = []
    for key, value in ordinates_dict.items():
        ordinates_list.append(value)
    
    road_lenght = 0
    index = 1
    for _ in ordinates_list[1:]:
        i, j = ordinates_list[index][0], ordinates_list[index][1]
        x, y = ordinates_list[index - 1][0], ordinates_list[index - 1][1]

        if (x - i == 10.0) and (y - j == -10.0):
            road_lenght += math.sqrt(2 * 100)
        else:
            road_lenght += 10 
    
    print(f'Road-length ALGO = [{namming_dict[p]}] = {road_lenght / 1000} kms')
    kilometers.append(road_lenght / 1000)

Road-length ALGO = [A* Chebyshev distance] = 150.44403876522486 kms
Road-length ALGO = [A* Diagonal distance] = 147.03578407990688 kms
Road-length ALGO = [A* Euclidean distance] = 147.1630633005204 kms
Road-length ALGO = [A* Manhattan distance] = 143.17498105462965 kms
Road-length ALGO = [A* Proposed] = 131.49357702943195 kms
Road-length ALGO = [Dijkstra's algorithm] = 136.02 kms


In [51]:
# reading LULC data
area_cover = 'mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

mask_data = area
print(mask_data.shape)
print(np.unique(mask_data, return_counts = True))

mask_data = mask_data.T

(20976, 20976)
(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int16), array([       14,   4922728,   5469572,  24929240,  47784093, 123283586,
       214883874,  18719469]))


In [52]:
for k in range(1, 7):

    path = paths_dict[k]
    path_list = []
    with open(f'OUTPUTS/{path}/path.txt', 'r') as f:
        for point in f:
            path_list.append(point)
    
    # print(len(path_list))
    path_list.reverse()
    classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}

    pixels_dict = {}
    for i in range(len(path_list)):
        point = path_list[i].replace('\n', '').split(' ')
        pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

        ext_i, ext_j = left + (pi * 10), top - (pj * 10)

        # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
        # ordinates_dict[pi, pj] = [ext_i, ext_j]
        pixels_dict[ext_i, ext_j] = [pi, pj]

    check_pixels = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
        8: 0,
    }

    df_dict_pixels = {}

    r = shapefile.Reader(f'OUTPUTS/{shape_dict[k]}/shapefiles/test/multipoint.shp')
    feature = r.shapeRecords()[0]
    first = feature.shape.__geo_interface__  

    total_pxs = len(first['coordinates'])
    for cord in first['coordinates']:
        pi, pj = pixels_dict[cord]
        cs = mask_data[pi][pj]

        check_pixels[cs] += 1
        check_pixels[8] += 1
    
    for key, value in check_pixels.items():
        df_dict_pixels[classes[key]] = f'{value} pxs'

    str = ''
    for i in range(165):
        if i % 2 == 0:
            str += '-'
        else:
            str += '#'
    
    print(str)

    print(f'ALGORITHM = [{namming_dict[k]}]')
    # print(f'PIXELS LULC {check_pixels}')
    # print(f'PIXELS LULC {df_dict_pixels}')

    for key, value in check_pixels.items():
        check_pixels[key] = np.round(((value / total_pxs) * 100), 2)
    
    # print(f'PERCENTAGE LULC {check_pixels}')

    classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}
    df_dict_perc = {}
    for key, value in check_pixels.items():
        df_dict_perc[classes[key]] = f'{check_pixels[key]}%'
    
    # print(f'ROAD PERCENTAGE = {df_dict_perc}')

    road_distance = MAIN_KILO
    current_road_distance = kilometers[k - 1]
    # print(f'CURRENT DISTANCE = {road_distance}, OPTIMIZED DISTANCE = {kilometers[k - 1]}')

    df_dict_dist = {}
    recheck = 0
    for key, value in check_pixels.items():
        df_dict_dist[classes[key]] = f'{(np.round(check_pixels[key] * current_road_distance) / 100)} Kms'
        recheck += (check_pixels[key] * road_distance) / 100
    
    # print(f'ROAD KILOMETERS = {df_dict_dist}')
    str = ''
    for i in range(165):
        if i % 2 == 0:
            str += '-'
        else:
            str += '#'
    
    list_of_dicts = [df_dict_pixels, df_dict_perc, df_dict_dist,]
    route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])
    display(route_stats)

    print(f'CURRENT DISTANCE = {np.round(road_distance)} Kms, OPTIMIZED DISTANCE = {kilometers[k - 1]} Kms')
    
    print(str)
            

-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Chebyshev distance]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,239 pxs,66 pxs,1770 pxs,3761 pxs,2108 pxs,1873 pxs,822 pxs,10639 pxs
ROAD PERCENTAGE,2.25%,0.62%,16.64%,35.35%,19.81%,17.61%,7.73%,100.0%
ROAD KILOMETERS,3.38 Kms,0.93 Kms,25.03 Kms,53.18 Kms,29.8 Kms,26.49 Kms,11.63 Kms,150.44 Kms


CURRENT DISTANCE = 206.0 Kms, OPTIMIZED DISTANCE = 150.44403876522486 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Diagonal distance]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,84 pxs,217 pxs,952 pxs,3560 pxs,2820 pxs,1576 pxs,1189 pxs,10398 pxs
ROAD PERCENTAGE,0.81%,2.09%,9.16%,34.24%,27.12%,15.16%,11.43%,100.0%
ROAD KILOMETERS,1.19 Kms,3.07 Kms,13.47 Kms,50.35 Kms,39.88 Kms,22.29 Kms,16.81 Kms,147.04 Kms


CURRENT DISTANCE = 206.0 Kms, OPTIMIZED DISTANCE = 147.03578407990688 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Euclidean distance]


...

CURRENT DISTANCE = 206.0 Kms, OPTIMIZED DISTANCE = 143.17498105462965 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Proposed]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,47 pxs,94 pxs,501 pxs,2180 pxs,2838 pxs,2779 pxs,860 pxs,9299 pxs
ROAD PERCENTAGE,0.51%,1.01%,5.39%,23.44%,30.52%,29.88%,9.25%,100.0%
ROAD KILOMETERS,0.67 Kms,1.33 Kms,7.09 Kms,30.82 Kms,40.13 Kms,39.29 Kms,12.16 Kms,131.49 Kms


CURRENT DISTANCE = 206.0 Kms, OPTIMIZED DISTANCE = 131.49357702943195 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [Dijkstra's algorithm]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,58 pxs,11 pxs,1550 pxs,10814 pxs,87 pxs,88 pxs,995 pxs,13603 pxs
ROAD PERCENTAGE,0.43%,0.08%,11.39%,79.5%,0.64%,0.65%,7.31%,100.0%
ROAD KILOMETERS,0.65 Kms,0.12 Kms,17.14 Kms,119.6 Kms,0.96 Kms,0.98 Kms,11.0 Kms,150.44 Kms


CURRENT DISTANCE = 206.0 Kms, OPTIMIZED DISTANCE = 150.44403876522486 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-


In [98]:
# list_of_dicts = [
#     df_dict_pixels,
#     df_dict_perc,
#     df_dict_dist,
# ]

In [104]:
# route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])

In [107]:
# route_stats

,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland
PIXELS LULC,367 pxs,143 pxs,4622 pxs,9025 pxs,1278 pxs,417 pxs,2386 pxs
ROAD PERCENTAGE,2.01%,0.78%,25.34%,49.48%,7.01%,2.29%,13.08%
ROAD KILOMETERS,4.7 Kms,1.83 Kms,59.3 Kms,115.79 Kms,16.4 Kms,5.36 Kms,30.61 Kms
